# ЛР8 (дополнительная). Алгоритм Витерби (HMM)

## 1) Что дано (по рисунку)

Скрытые состояния (hidden states):
1. **Food street**  (обозначим F)
2. **Clothes market** (обозначим C)
3. **Electronics market** (обозначим E)

Наблюдения (observations):
- **Food**
- **Clothes**
- **Electronics**

### Начальные вероятности π (Initial state → {F,C,E})
По рисунку: 0.33, 0.33, 0.33  
(из-за округления сумма 0.99, принимаем как задано)

\[
\pi = [0.33,\;0.33,\;0.33]
\]

### Матрица переходов A (между скрытыми состояниями)
Считываем с рисунка:
- F→F = 0.1, F→C = 0.6, F→E = 0.3
- C→F = 0.45, C→C = 0.15, C→E = 0.4
- E→F = 0.4, E→C = 0.4, E→E = 0.2

Порядок состояний: [F, C, E]

\[
A =
\begin{pmatrix}
0.10 & 0.60 & 0.30 \\
0.45 & 0.15 & 0.40 \\
0.40 & 0.40 & 0.20
\end{pmatrix}
\]

### Матрица эмиссий B (скрытое состояние → наблюдение)
Считываем пунктирные стрелки:
- из F: P(Food)=0.7, P(Clothes)=0.2, P(Electronics)=0.1
- из C: P(Food)=0.25, P(Clothes)=0.65, P(Electronics)=0.1
- из E: P(Food)=0.15, P(Clothes)=0.1, P(Electronics)=0.75

Порядок наблюдений: [Food, Clothes, Electronics]

\[
B =
\begin{pmatrix}
0.70 & 0.20 & 0.10 \\
0.25 & 0.65 & 0.10 \\
0.15 & 0.10 & 0.75
\end{pmatrix}
\]

## 2) Последовательность наблюдений
Возьмём последовательность длины 3 (как в классическом примере из статьи):
\[
O = [Food,\;Clothes,\;Electronics]
\]


In [1]:
import numpy as np
import pandas as pd

# Состояния (hidden)
states = ["Food street (F)", "Clothes market (C)", "Electronics market (E)"]

# Наблюдения (observations)
observations = ["Food", "Clothes", "Electronics"]

# π (начальные вероятности)
pi = np.array([0.33, 0.33, 0.33])

# A (матрица переходов), порядок [F, C, E]
A = np.array([
    [0.10, 0.60, 0.30],  # from F
    [0.45, 0.15, 0.40],  # from C
    [0.40, 0.40, 0.20]   # from E
])

# B (матрица эмиссий), строки = [F, C, E], столбцы = [Food, Clothes, Electronics]
B = np.array([
    [0.70, 0.20, 0.10],  # state F
    [0.25, 0.65, 0.10],  # state C
    [0.15, 0.10, 0.75]   # state E
])

# Последовательность наблюдений O = [Food, Clothes, Electronics]
O = ["Food", "Clothes", "Electronics"]
O_idx = [observations.index(x) for x in O]

pi, A, B, O_idx

(array([0.33, 0.33, 0.33]),
 array([[0.1 , 0.6 , 0.3 ],
        [0.45, 0.15, 0.4 ],
        [0.4 , 0.4 , 0.2 ]]),
 array([[0.7 , 0.2 , 0.1 ],
        [0.25, 0.65, 0.1 ],
        [0.15, 0.1 , 0.75]]),
 [0, 1, 2])

In [2]:
T = len(O_idx)
N = len(states)

delta = np.zeros((T, N), dtype=float)
psi = np.full((T, N), -1, dtype=int)

# t=0
delta[0, :] = pi * B[:, O_idx[0]]

# t=1..T-1
for t in range(1, T):
    for j in range(N):
        candidates = delta[t-1, :] * A[:, j]          # все варианты прийти в j
        psi[t, j] = int(np.argmax(candidates))        # откуда пришли
        delta[t, j] = np.max(candidates) * B[j, O_idx[t]]

# Красивый вывод
delta_df = pd.DataFrame(delta, columns=states)
psi_df = pd.DataFrame(psi, columns=states)

print("Наблюдения O =", O)
print("\nDELTA (вероятности):")
display(delta_df)

print("\nPSI (указатели откуда пришли, индексы состояний):")
display(psi_df)

# Финал: лучший последний стейт
last_state = int(np.argmax(delta[T-1, :]))
best_prob = float(np.max(delta[T-1, :]))

# Backtracking
path_idx = [last_state]
for t in range(T-1, 0, -1):
    path_idx.append(psi[t, path_idx[-1]])
path_idx = list(reversed(path_idx))

best_path = [states[i] for i in path_idx]

print("\nЛУЧШАЯ СКРЫТАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ:")
for t, s in enumerate(best_path):
    print(f"t={t}, obs={O[t]}  ->  hidden={s}")

print(f"\nМаксимальная вероятность пути P* = {best_prob}")


Наблюдения O = ['Food', 'Clothes', 'Electronics']

DELTA (вероятности):


,Food street (F),Clothes market (C),Electronics market (E)
0,0.231000,0.082500,0.049500
1,0.007425,0.090090,0.006930
2,0.004054,0.001351,0.027027



PSI (указатели откуда пришли, индексы состояний):


,Food street (F),Clothes market (C),Electronics market (E)
0,-1,-1,-1
1,1,0,0
2,1,1,1



ЛУЧШАЯ СКРЫТАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ:
t=0, obs=Food  ->  hidden=Food street (F)
t=1, obs=Clothes  ->  hidden=Clothes market (C)
t=2, obs=Electronics  ->  hidden=Electronics market (E)

Максимальная вероятность пути P* = 0.027027
